# Overview

In this tutorial we will introduce key concepts related to creating simple convolutional neural networks.

**Convolutional Neural Networks**

* convolutional operations
* softmax cross-entroy loss
* model training
* evaluation
* visualization

This tutorial is part of the class **Introduction to Deep Learning for Medical Imaging** at University of California Irvine (CS190); more information can be found at: https://github.com/peterchang77/dl_tutor/tree/master/cs190.

# Google Colab

### Enable GPU runtime

Use the following instructions to switch the default Colab instance into a GPU-enabled runtime:

```
Runtime > Change runtime type > Hardware accelerator > GPU
```

# Environment

### Jarvis library

In this notebook we will Jarvis, a custom Python package to facilitate data science and deep learning for healthcare. Among other things, this library will be used for low-level data management, stratification and visualization of high-dimensional medical data.

In [ ]:
# --- Install jarvis (only in Google Colab or local runtime)
% pip install jarvis-md

### Imports

Use the following lines to import any additional needed libraries:

In [ ]:
import numpy as np, pandas as pd
from tensorflow.keras import Input, Model, models, layers, losses, metrics, optimizers
from jarvis.train import datasets
from jarvis.utils.display import imshow

# Data

The data used in this tutorial will consist of the CIFAR-10 dataset comprising 10 different everyday objects (airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck). The custom `datasets.download(...)` method can be used to download a local copy of the dataset. By default the dataset will be archived at `/data/raw/cifar`; as needed an alternate location may be specified using `datasets.download(name=..., path=...)`. 

In [ ]:
# --- Download dataset
datasets.download(name='cifar')

Once downloaded, the `datasets.prepare(...)` method can be used to generate the required python Generators to iterate through the dataset, as well as a `client` object for any needed advanced functionality. As needed, pass any custom configurations (e.g. batch size, normalization parameters, etc) into the optional `configs` dictionary argument.

In [ ]:
# --- Prepare generators
configs = {'batch': {'size': 36}}
gen_train, gen_valid, client = datasets.prepare(name='cifar', configs=configs, custom_layers=True)

The created generators yield a total of `batch['size']` training samples based on the specified batch size. As before, each iteration yields dictionary of model inputs, `xs`. In the current example, there is just a single input image `xs['dat']` and a single target `xs['class']`. Let us examine the generator data: Let us examine the generator data:

In [ ]:
# --- Yield one example
xs, _ = next(gen_train)

# --- Print dict keys
for k, v in xs.items():
    print('key = {} : shape = {}'.format(k.ljust(7), v.shape))

Use the following lines of code to visualize using the `imshow(...)` method:

In [ ]:
# --- Show the first example
imshow(xs['dat'][0])

Use the `montage(...)` function to create an N x N mosaic of all images:

In [ ]:
# --- Show "montage" of all images
imshow(xs['dat'])

As expected, the 36-element `xs['class']` vector corresponds to ground-truth:

In [ ]:
# --- Print xs['class']
print(xs['class'].reshape(6, 6))

# Convolutional Operations

In this tutorial, a CNN will be created using 2D convolutional operations. Recall that convolutional operations are defined by the following minimum specifications:

* filter / channel depth
* kernel size
* strides
* padding

Let us first begin by defining a single `Input` variable `x`:

In [ ]:
# --- Define model input 
x = Input(shape=(32, 32, 3), dtype='float32')

To apply convolutional operations to this input tensor `x`, consider the following:

In [ ]:
# --- Define regular convolution
l1 = layers.Conv2D(
    filters=16, 
    kernel_size=(3, 3), 
    strides=(1, 1), 
    padding='same')(x)

# --- Define strided convolution
l1 = layers.Conv2D(
    filters=16, 
    kernel_size=(3, 3), 
    strides=(2, 2), 
    padding='same')(x)

To reuse identical function arguments, consider maintaining a `kwargs` dictionary and pass using the `**` symbol:

In [ ]:
# --- Define kwargs dictionary
kwargs = {
    'kernel_size': (3, 3),
    'padding': 'same'}

# ---- Define stack of convolutions
l1 = layers.Conv2D(filters=16, strides=(1, 1), **kwargs)(x)
l2 = layers.Conv2D(filters=32, strides=(1, 1), **kwargs)(l1)

## Blocks

In addition to the requisite convolutional operationss and activation functions, batch normalization is almost universally used in modern CNN architectures. Thus at minimum, a common baseline *block* pattern of operations can be defined as:

* convolutional operation
* batch normalization
* activation function (e.g. ReLU)

Let us define a block:

In [ ]:
# --- Define block
c1 = layers.Conv2D(filters=16, **kwargs)(x)
n1 = layers.BatchNormalization()(c1)
r1 = layers.ReLU()(n1)

During the course of buildling CNNs, we will be writing **many** blocks over time. Thus for brevity, let us use lambda functions to define modular, reusable components:

In [ ]:
# --- Define lambda functions
conv = lambda x, filters, strides : layers.Conv2D(filters=filters, strides=strides, **kwargs)(x)
norm = lambda x : layers.BatchNormalization()(x)
relu = lambda x : layers.ReLU()(x)

Great! Now let us rewrite a block using lambda shorthand:

In [ ]:
# --- Define block
b1 = relu(norm(conv(x, 16, (1, 1))))

In practice, the two most common **block patterns** will be regular convolutional block and a strided convolutional block (for subsampling). Let us then create two more high-level lambda functions for this:

In [ ]:
# --- Define stride-1, stride-2 blocks
conv1 = lambda filters, x : relu(norm(conv(x, filters, strides=1)))
conv2 = lambda filters, x : relu(norm(conv(x, filters, strides=(2, 2))))

Now let us see how easy it is to create series of alternating stride-1 and stride-2 blocks:

In [ ]:
# --- Define series of blocks
l1 = conv1(16, x)
l2 = conv1(24, conv2(24, l1))
l3 = conv1(32, conv2(32, l2))
l4 = conv1(48, conv2(48, l3))

# Fully Connected Layers

After a series of convolutional block operations, recall that the CNN must transition to MLP type operations (e.g. matrix multiplications). To convert 3D (or 4D) feature maps into vectors, consider one of the following approaches:

* serial convolutions (with stride > 1 or VALID type padding)
* global pool operations (mean or max)
* reshape / flatten operation

The following line of code implements the reshape / flatten operation:

In [ ]:
f0 = layers.Flatten()(l4)

The other options are described in further detail in the tutorial exercises.

### MLP

Now that the intermediate layer is defined as a vector, a number of standard MLP type operations may be performed, including creation of an arbitrary (optional) number of hidden layers. As before, the final layer should yield a vector of logit scores **without** any activation function applied.

In [ ]:
# --- Add hidden layers if needed
# .... (optional) ...

# --- Final logit scores
logits  = layers.Dense(10, name='class')(f0)

# Creating Models

## Backbone model

Using the concepts introduced above, let us start by recreating the base `backbone` model (e.g., without training dependencies). Use the following line of code to create the `backbone` model:

In [ ]:
# --- Define input
x = Input(shape=(32, 32, 3), dtype='float32')

# --- Define series of blocks
l1 = conv1(16, x)
l2 = conv1(24, conv2(24, l1))
l3 = conv1(32, conv2(32, l2))
l4 = conv1(48, conv2(48, l3))

# --- Flatten
f0 = layers.Flatten()(l4)

# --- Final logit scores
logits  = layers.Dense(10, name='class')(f0)

# --- Create model
backbone = Model(inputs=x, outputs=logits)

Use the following line of code to pass a single batch of data through the created model:

In [ ]:
# --- Predict
scores = backbone.predict(xs['dat'])

What is the output shape of the `scores` variable, what does this variable represent?

In [ ]:
# --- Logit scores
print(scores.shape)

## Training model

Next, the `backbone` model architecture is wrapped in a second model with additional layer(s) that define optimization behavior including loss function derivations.

### Inputs

As before, we start by defining all `inputs` into our new *wrapper* model. In the first model, we only defined a single `x` input because the `backbone` base model is designed to accept any input value(s) and make predictions without requiring a target. In our second *wrapper* model, specifically designed to facilitate model training, we additionally need an `Input` object for the target `class` so that we can calculate a loss value and improve our model performance through optimization. 

In [ ]:
inputs = {
    'dat': Input(shape=(32, 32, 3), name='dat'),
    'class': Input(shape=(1,), name='class')}

Using this `inputs` Python dictionary, let us first recreate the CNN model operations by **reusing** the `backbone` object that we already defined. Doing so means that our new *wrapper* model is explicitly derived from the `backbone`. Any updates applied to our new *wrapper* model are propogated to the `backbone` model and vice versa. 

In [ ]:
# --- Define first step of new wrapper model
logits = backbone(inputs['dat'])

### Softmax cross-entropy loss

For a classification problem, the most common loss is defined by **softmax cross-entropy**. This loss in fact is composed of two seperate steps:

* **softmax**: method to normalize logit scores into a probability distribution
* **cross-entropy**: method to quantify difference between predicted distribution and true distribution

To define a softmax cross-entropy loss in Tensorflow, use the `losses.SparseCategoricalCrossentropy(...)` class. 

In [ ]:
# --- Define a loss object
sce = losses.SparseCategoricalCrossentropy(from_logits=True)

The returned variable `sce` is a Python method that can be used to calculate softmax cross-entropy loss for a given set of logit scores (`y_pred`) and ground-truth (`y_true`). For demonstration, use the following lines of code to calculate the loss for the generate logit scores of the current data batch:

In [ ]:
# --- Calculate loss
sce(y_true=xs['class'], y_pred=scores)

As training proceeds, the goal of the algorithm is to update weights in a manner such that this loss value decreases over time.

The `scores` variable represents the logit scores for one batch of input data. However, we still need to define a symbolic TensorFlow tensor as a part of our graph. Let us formally create this loss tensor now using the `sce` function using the `logits` returned by the `backbone` call earlier.

In [ ]:
# --- Create loss tensor
loss = sce(y_true=inputs['class'], y_pred=logits)

### Accuracy metric

Compared to *loss* values which the model will directly optimize, a *metric* is a quantitative value that captures a more human-interpretable value for overall model performance. Because the metric does not contribute to the optimization process (only used to monitor performance), this value **does not** need to be differentiable.

For a classification problem, the most common metric is overall *accuracy*. Use the TensorFlow built-in method `metrics.sparse_categorical_accuracy(...)` to calculate per-batch accuracy:

In [ ]:
# --- Define accuracy
acc = metrics.sparse_categorical_accuracy(y_true=inputs['class'], y_pred=logits)

### Create model

Now let us create the new wrapper model. The inputs are defined above already in our `inputs` Python dictionary. As outputs, let us return both the `logits` tensor as well as the `loss`. We will name this new wrapper model `training` because it will be used for training only. 

In [ ]:
training = Model(inputs=inputs, outputs={'logits': logits, 'loss': loss, 'acc': acc})

Now let's add the `loss` and `metric` tensorws we defined above to the new `training` model:

In [ ]:
# --- Add loss
training.add_loss(loss)

# --- Add metric
training.add_metric(acc, name='acc')

## Compiling a Model

The current model parameters have been initialized to random values. Through exposure to data, the goal is for the model to *learn* optimal parameter values that allow for robust mapping of provided input to target output. To prepare the model for learning, a graph must be **compiled** with a strategy for optimization.

In [ ]:
# --- Define an Adam optimizer
optimizer = optimizers.Adam(learning_rate=2e-4)

# --- Compile model
training.compile(optimizer=optimizer)

# Model Training

Once the model has been compiled and the data prepared (via a generator), training can be invoked using the `training.fit(...)` method. Ensure that both the training and validation data generators are used. In this particular example, we are defining arbitrary epochs of 250 steps each. Training will proceed for 12 epochs in total. Validation statistics will be assess every fourth epoch. As needed, tune these arugments as need.

In [ ]:
training.fit(
    x=gen_train, 
    steps_per_epoch=250, 
    epochs=12,
    validation_data=gen_valid,
    validation_steps=250,
    validation_freq=4)

### Evaluation

To test the trained model, the following steps are required:

* load data
* use `backbone.predict(...)` or `training.predict(...)` to obtain logit scores
* use `np.argmax(...)` to obtain prediction
* compare prediction with ground-truth
* serialize in Pandas DataFrame

Recall that the generator used to train the model simply iterates through the dataset randomly. For model evaluation, the cohort must instead be loaded manually in an orderly way. For this tutorial, we will create new **test mode** data generators, which will simply load each example individually once for testing. 

In [ ]:
# --- Create validation generator
test_train, test_valid = client.create_generators(test=True)

Use the following lines of code to yield all examples from the valid generator:

In [ ]:
# --- Aggregate all examples
xs = {'dat': [], 'class': []}

for x, _ in test_valid:
    xs['dat'].append(x['dat'])
    xs['class'].append(x['class'])

xs = {
    'dat': np.concatenate(xs['dat']),
    'class': np.concatenate(xs['class'])}

Use the following lines of code to run prediction through the **valid** cohort generator:

In [ ]:
# --- Predict
outputs = training.predict(xs)

# --- Argmax
pred = np.argmax(outputs['logits'], axis=1)

Prepare results in Pandas DataFrame for ease of analysis and sharing:

In [ ]:
df = pd.DataFrame(index=np.arange(pred.size))

# --- Define columns
df['true'] = xs['class']
df['pred'] = pred
df['corr'] = df['true'] == df['pred']

# --- Print accuracy
print(df['corr'].mean())

How does this compare to the `accuracy` calculated by `training.predict(...)`?

## Saving and Loading a Model

After a model has been successfully trained, it can be saved and/or loaded by simply using the `backbone.save()` and `backbone.load_model()` methods. 

In [ ]:
# --- Serialize a model
backbone.save('./model.hdf5')

In [ ]:
# --- Load a serialized model
del backbone
backbone = models.load_model('./model.hdf5', compile=False)

# Exercises

Up until this point, this tutorial has presented a baseline of minimum requirements needed to successful train a CNN. The following exercises will assist in further optimization and customization of network architecture as well as hyperparameter considerations. Upon completion, apply any combination of techniques explored in this tutorial to finish the assignment: to train a CNN on the CIFAR-10 dataset.

### Exercise 1

In addition a matrix flatten operation, conversion from 3D (or 4D) feature maps into vectors may be accomplished via:

* serial convolutions (with stride > 1 or VALID type padding)
* global pool operations (mean or max)

Use the following cell to implement this two alternative methods:

In [ ]:
# --- Define input
x = Input(shape=(32, 32, 3), dtype='float32')

# --- Define series of blocks
l1 = conv1(16, x)
l2 = conv1(24, conv2(24, l1))
l3 = conv1(32, conv2(32, l2))
l4 = conv1(48, conv2(48, l3))

# --- Convert l4 to vector via serial convolution(s)
op = layers.Conv2D(
    filters=48, 
    kernel_size=?, 
    strides=?, 
    padding=?)

vector = ?

# --- Contert l4 to vector via global pool operations
op = layers.?
vector = ?

#### Hints

To implement via serial convolution(s), consider the size of the last feature map layer. Why is this size important? Accordiningly, what size must the convolutional filter be to accomplish the goal of generating a `1 x 1 x C` feature map? In addition to changes in convolutional `kernel_size`, how to modifications to `stride` and `padding` effect the output layer size?

Use the following cell to experiment:

In [ ]:
# --- Check l4 size
l4.shape

# --- Alter convolutional kernel size

# --- Alter convolutional stride

# --- Alter convolutional padding

To implement via global pooling operations, consider referencing the Keras API here: https://keras.io/api/layers/pooling_layers/.

### Exercise 2

To limit overfitting, consider use of an L2 regularizer. Recall that this technique adds an extra term to the loss function such that in addition to optimizing for classification accuracy, the model also will attempt to minimize overall L2 distance of all network parameters. 

In the Tensorflow 2 / Keras API, a `kernel_reguarlizer` can be specified in the definition of any layer operation: 

In [ ]:
from tensorflow.keras import regularizers

# --- Use regularizer
op = layers.Conv2D(
    filters=16, 
    kernel_size=(3, 3), 
    strides=(1, 1), 
    padding='same',
    kernel_regularizer=regularizers.l2(0.01))

l1 = relu(norm(op(x)))

The *lambda* parameter passed into the `regularizers.l2(...)` call is a constant value by which to scale this component of the loss; for more L2 regularization, use higher values. A *lambda* value of 0 is equivalent to no regularizer use at all.

#### Hints

What other types of regularizers are present in the `regularizers.*` module? Can L1, L2 and/or both L1/L2 be used? What would be the relative benefit of some of these alternatives?

### Exercise 3

To limit overfitting, consider use of Dropout. The `rate` argument passed during initialization determines the **fraction of input units to drop**. Thus a rate of 0.25 will retain 75% of activation values.

In [ ]:
# --- Define dropout
drop = layers.Dropout(rate=0.25)

When using the ReLU activation function, dropout can be applied either before or after the activation function (otherwise it is most common to apply dropout after the activation function). Also recall that dropout is most efficient in the MLP portions of your network. Finally just as the final matrix multiplication should not be defined with any activation function, ensure the **no dropout** is used in the final layer either.

In [ ]:
# --- Define series of blocks
l1 = conv1(16, x)
l2 = conv1(24, conv2(24, l1))
l3 = conv1(32, conv2(32, l2))
l4 = conv1(48, conv2(48, l3))

# --- Flatten
f0 = layers.Flatten()(l4)

# --- Use dropout
hidden_size = 64
f1 = drop(layers.Dense(hidden_size, activation='relu')(f0))